In [1]:
try:
    import binutil
except ModuleNotFoundError:
    import bin.binutil

from dreamcoder.program import *
from dreamcoder.domains.relation import *
from dreamcoder.domains.relation.relation_primitives import *

get_baseline_primitives()
get_clevr_primitives()

DEPRECATION NOTICE: this module (dreamcoder.ec) will be deleted soon, please update your code to import from dreamcoder.dreamcoder instead


In [21]:
# read txt file
mode = "test"
split = "length_threshold"  # material
same_support = True
seed = 2

if same_support:
    domain = "clevr"
    data_folder = "curi_dc_test_tasks"
else:
    domain = "clevr_single_support"  # "clevr_single_support"
    data_folder = "curi_dc_test_tasks_single"  # "curi_dc_test_tasks_single"

file = f"../consoleOutputs/{domain}/{split}/eval/{seed}/{mode}_programs.out"

with open(file, "r") as f:
    data = f.read()

if mode == "train":
    data = data.split("\n")
    data = [line for line in data if line != ""]
    task_programs = {}
    task_flag = False
    for line in data:
        if task_flag:
            program = line.split("\t")[1]
            task_programs[task_id] = program
            task_flag = False
        if "task" in line:
            task_flag = True
            task_id = line
else:
    data = data.split("\n")
    NUMBER_TEST_TASKS = len(data)
    print("Number of all tasks: ", len(data))
    task_programs = {}
    for task in data:
        if not "HIT" in task:
            continue
        task = task.split("w/")
        name = task[0].replace(" ", "")
        name = name.replace("HIT", "")
        program = task[1].split(";")[0][1:]
        task_programs[name] = program

Number of all tasks:  8363


In [22]:
import pandas as pd
import json
import os
from tqdm import tqdm

path = f"../data/{data_folder}/{split}/query/"
json_files = [f.path for f in os.scandir(path) if f.path.endswith(".json")]

task_results = []

# iterate over tasks
for task_file in tqdm(json_files):
    task_name = task_file.split("/")[-1][:-5]
    # task_number = task_name.split("_")[-1]
    # check if program exists for task
    if task_name in task_programs.keys():
        f = open(task_file)
        examples = json.load(f)

        parsed_examples = []

        true_positives = 0
        false_positives = 0
        true_negatives = 0
        false_negatives = 0

        for example in examples:
            input = example["input"]
            output = example["output"]

            # execute program for input
            program = task_programs[task_name]
            program = Program.parse(program)
            try:
                program_output = program.evaluate([])(input)
            except:
                continue

            # categorize prediction
            if output:
                if program_output:
                    true_positives += 1
                else:
                    false_negatives += 1
            else:
                if program_output:
                    false_positives += 1
                else:
                    true_negatives += 1

        # collect results
        results = {
            "task_name": task_name,
            "program": task_programs[task_name],
            "TP": true_positives,
            "FP": false_positives,
            "TN": true_negatives,
            "FN": false_negatives,
        }
        task_results.append(results)
    else:
        continue
        results = {"task_name": task_name, "TP": 0, "FP": 0, "TN": 0, "FN": 0}
        task_results.append(results)

results_df = pd.DataFrame(task_results)
results_df

  0%|          | 0/8363 [00:00<?, ?it/s]

100%|██████████| 8363/8363 [03:54<00:00, 35.67it/s]


,task_name,program,TP,FP,TN,FN
0,task_10727,(lambda (#(lambda (lambda (#(lambda (lambda (l...,1,1,19,4
1,task_6538,(lambda (#(lambda (lambda (lambda (eq? $1 (cou...,5,4,36,5
2,task_3467,(lambda (#(lambda (lambda (exists (lambda ($1 ...,15,0,60,0
3,task_7390,(lambda (#(lambda (lambda (forall (lambda (eq?...,5,0,20,0
4,task_1982,(lambda (#(lambda (lambda (lambda (#(lambda (l...,5,8,12,0
...,...,...,...,...,...,...
6909,task_8684,(lambda (#(lambda (lambda (lambda (eq? $1 (cou...,0,3,17,5
6910,task_9811,(lambda (#(lambda (lambda (eq? (#(lambda (coun...,4,0,20,1
6911,task_2013,(lambda (#(lambda (lambda (lambda (eq? $1 (cou...,1,1,19,4
6912,task_2449,(lambda (#(lambda (lambda (lambda (eq? $1 (cou...,5,0,20,0


In [23]:
results_df["Accuracy"] = (results_df["TP"] + results_df["TN"]) / (
    results_df["TP"] + results_df["TN"] + results_df["FP"] + results_df["FN"]
)
results_df["CBA"] = (
    (results_df["TP"] / (results_df["TP"] + results_df["FN"]))
    + (results_df["TN"] / (results_df["TN"] + results_df["FP"]))
) / 2
results_df["Precision"] = results_df["TP"] / (results_df["TP"] + results_df["FP"])
# get number of tasks with accuracy not nan
len(results_df[~results_df["Accuracy"].isna()])

6914

In [24]:
results_df.sort_values(by="CBA", ascending=True)

,task_name,program,TP,FP,TN,FN,Accuracy,CBA,Precision
4962,task_12533,(lambda (#(lambda (lambda (exists (lambda ($1 ...,0,12,8,5,0.32,0.2000,0.000000
6255,task_6773,(lambda (#(lambda (lambda (lambda (eq? $1 (cou...,0,12,8,5,0.32,0.2000,0.000000
822,task_14598,(lambda (forall (lambda (#(lambda (lambda (for...,3,35,5,7,0.16,0.2125,0.078947
5014,task_13203,(lambda (#(lambda (lambda (#(lambda (lambda (l...,0,11,9,5,0.36,0.2250,0.000000
4915,task_1371,(lambda (#(lambda (lambda (lambda (#(lambda (e...,0,11,9,5,0.36,0.2250,0.000000
...,...,...,...,...,...,...,...,...,...
441,task_4774,(lambda (#(lambda (lambda (forall (lambda (eq?...,15,0,60,0,1.00,1.0000,1.000000
3852,task_2261,(lambda (#(lambda (lambda (forall (lambda (eq?...,15,0,60,0,1.00,1.0000,1.000000
5383,task_1169,(lambda (#(lambda (lambda (gt? (count (map ($0...,5,0,20,0,1.00,1.0000,1.000000
5404,task_1950,(lambda (#(lambda (lambda (forall (lambda (eq?...,5,0,20,0,1.00,1.0000,1.000000


In [25]:
print(split, " ", mode)
# get mean accuracy
print(results_df["Accuracy"].mean())
# get mean CBA
print(results_df["CBA"].mean())
# get precision
print(results_df["Precision"].mean())

length_threshold   test
0.8412637544100809
0.7777779605508032
0.6402222518701884


In [26]:
print(split, " ", mode, " ", seed)
print("Class balanced accuracy (solved): ", results_df["CBA"].mean())

cba_50 = results_df["CBA"].mean() * (len(results_df) / NUMBER_TEST_TASKS) + 0.5 * (
    1 - (len(results_df) / NUMBER_TEST_TASKS)
)
# round to 2 decimal places
cba_50 = round(cba_50 * 100, 2)
print("Class balanced accuracy (all): ", cba_50)

length_threshold   test   2
Class balanced accuracy (solved):  0.7777779605508032
Class balanced accuracy (all):  72.96


In [27]:
# save results
results_df.to_csv(f"../experimentOutputs/{domain}/{split}_{mode}_image_{seed}.csv")
print(
    "saved results to ",
    f"../experimentOutputs/{domain}/{split}_{mode}_image_{seed}.csv",
)

saved results to  ../experimentOutputs/clevr/length_threshold_test_image_2.csv


In [28]:
# get @all accuracies
import pandas as pd
import numpy as np

domain = "clevr"
split = "length_threshold"
mode = "test"
NUMBER_TEST_TASKS = 8363

results = []
for seed in range(3):
    results_df = pd.read_csv(
        f"../experimentOutputs/{domain}/{split}_{mode}_image_{seed}.csv"
    )
    results.append(results_df)

mean_accs = [df["CBA"].mean() for df in results]
print(mean_accs)

cba_all = [0, 0, 0]
for seed in range(3):
    cba_all[seed] = mean_accs[seed] * (len(results[seed]) / NUMBER_TEST_TASKS) + 0.5 * (
        1 - (len(results[seed]) / NUMBER_TEST_TASKS)
    )

mean_accs = np.array(cba_all) * 100
print(mean_accs)
mean_acc = np.mean(mean_accs)
std = np.std(mean_accs)
print(round(mean_acc, 2), round(std, 2))

[0.769461059243057, 0.7755793038855253, 0.7777779605508032]
[71.71347696 72.44374792 72.96492669]
72.37 0.51


In [29]:
# get @solved accuracies
import pandas as pd
import numpy as np

# domain = "clevr"
# split = "color_material"
# mode = "test"

results = []
for seed in range(3):
    results_df = pd.read_csv(
        f"../experimentOutputs/{domain}/{split}_{mode}_image_{seed}.csv"
    )
    results.append(results_df)

mean_accs = [df["CBA"].mean() for df in results]
print(mean_accs)
mean_accs = np.array(mean_accs) * 100
mean_acc = np.mean(mean_accs)
std = np.std(mean_accs)
print(round(mean_acc, 2), round(std, 2))

[0.769461059243057, 0.7755793038855253, 0.7777779605508032]
77.43 0.35


In [21]:
# get programs with high accuracy
import pandas as pd
import json

split = "iid_sampling"
mode = "test"
file = f"../experimentOutputs/clevr/{split}_{mode}_0.csv"
results_df = pd.read_csv(file)

hyp_to_id = json.load(open("hyp_to_id.json"))


def get_hypothesis(id):
    for key, value in hyp_to_id.items():
        if value == int(id):
            return key


results_df["Hypothesis"] = results_df["task_name"].apply(
    lambda x: get_hypothesis(x.split("_")[-1])
)

example_results = results_df[results_df["Accuracy"] == 1]
example_results = example_results[["task_name", "Hypothesis", "program"]]
example_results

for index, row in example_results.iterrows():
    if "or " in row["Hypothesis"]:
        continue
    if "and " in row["Hypothesis"]:
        continue

    # if not "material" in row["Hypothesis"]:
    #     continue

    print(row["task_name"])
    print(row["Hypothesis"])
    print(row["program"])
    print()

task_7043
cube x shape? = lambda S. for-all=
(lambda (#(#(lambda (lambda (forall (lambda (eq? (index $2 $0) $1))))) 6 0) $0)) 

task_11438
x color? blue = lambda S. for-all=
(lambda (#(#(lambda (lambda (forall (lambda (eq? (index $2 $0) $1))))) 5) 1 $0)) 

task_5558
S shape? cylinder count= 3 = lambda S.
(lambda (eq? (#(lambda (count (map (lambda (#(lambda (lambda (+ (index $1 $0) 4))) 5 $0)) $0))) (map (lambda (cdr $0)) $0) 6) 3)) 

task_178
0.7 x size? = not lambda S. for-all=
(lambda (#(#(lambda (lambda (forall (lambda (eq? (index $2 $0) $1))))) 4 0) $0)) 

task_8572
non-x-S color? yellow for-all= lambda S. exists=
(lambda (eq? (#(lambda (count (map (lambda (#(lambda (lambda (+ (index $1 $0) 4))) 5 $0)) $0))) $0 7) (length (cdr $0)))) 

task_5732
S shape? cylinder for-all= lambda S.
(lambda (#(#(#(lambda (lambda (forall (lambda (eq? (index $2 $0) $1))))) 5) 2) (map (lambda (cdr $0)) $0))) 

task_2643
S locationX? 1 count= S size? x size? count= > lambda S. for-all=
(lambda (forall (